# Data Management with OpenACC

此实验面向 Fortran 程序员。如果您更喜欢使用 C/C++，请单击 [此链接](../C/README.ipynb)

不要忘记查看其他 [OpenACC 资源](https://www.openacc.org/resources) 并加入我们的 [OpenACC Slack 频道](https://www.openacc.org/community#slack)，以分享您的经验并从社区获得更多帮助。

---

让我们执行下面的单元格以显示有关服务器上运行的 GPU 的信息。为此，请执行下面的单元格块，方法是将焦点放在它上面（用鼠标单击它），然后按 Ctrl-Enter，或按上面工具栏中的播放按钮。如果一切顺利，您应该会看到灰色单元格下方返回一些输出。

In [ ]:
!nvidia-smi

---

## 简介

我们在本实验中的目标是使用 OpenACC 数据指令来正确管理我们的数据。我们将使用迄今为止使用的相同开发周期。即 3 步开发周期。

![development_cycle.png](../images/development_cycle.png)

这是 OpenACC 3 步开发周期。

**分析**您的代码，并预测可以发现潜在并行性的位置。使用分析器帮助了解代码中正在发生的事情以及可能存在并行性的位置。

**并行化**您的代码，从最耗时的部分开始。专注于维护程序的正确结果。

**优化**您的代码，专注于最大化性能。在早期并行化期间，性能可能不会一次性提高。

我们目前正在处理**并行化**步骤。我们将包括 OpenACC 数据指令，以便在并行化代码中正确管理数据。

---

## 运行代码（使用托管内存）

在上一个实验中，我们使用 CUDA 托管内存运行代码，并实现了显著的性能提升。但是，托管内存并不兼容所有 GPU，并且其性能可能比程序员定义的正确内存管理更差。运行以下脚本，并记下程序运行所需的时间。我们预计我们稍后将在本实验中开发的自己的实现将运行得更好一些。

In [ ]:
!nvfortran -fast -ta=tesla:managed -Minfo=accel -o laplace_managed laplace2d.f90 jacobi.f90 && ./laplace_managed

### 可选：查看代码

如果您想复习一下我们正在处理的代码文件，您可以使用下面的两个链接查看它们。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

### 可选：分析代码

In [ ]:
!nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o laplace_managed ./laplace_managed

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](laplace_managed.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看分析器报告，请参阅 [如何查看报告](../../../module2/assets/Fortran/README.ipynb) 部分。

---

## OpenACC 结构化数据指令

OpenACC 数据指令允许程序员明确管理设备（在我们的例子中是 GPU）上的数据。具体来说，结构化数据指令将把我们代码的静态区域标记为**数据区域**。

```fortran
< Initialize data on host (CPU) >

!$acc data clauses

	< Sequential and/or Parallel code >

!$acc end data
```

设备内存分配发生在区域的开始处，而设备内存释放发生在区域的结束处。此外，任何从主机到设备（CPU 到 GPU）的数据移动都发生在区域的开始处，任何从设备到主机（GPU 到 CPU）的数据移动都发生在区域的结束处。内存分配/释放和数据移动由程序员包含的子句定义。以下是我们可以使用的最重要数据子句的列表：

**copy** : `copy( A(1:N) )`：在进入区域时在设备上分配内存并将数据从主机复制到设备，在退出区域时将数据复制回主机

**copyin** : `copyin( A(1:N) )`：在进入区域时在设备上分配内存并将数据从主机复制到设备

**copyout** : `copyout( A(1:N) )`：在退出区域时在设备上分配内存并将数据复制到主机

**create** : `create( A(1:N) )`：在设备上分配内存但不复制

**present** : `present( A )`：数据已存在于另一个包含数据的区域的设备上

所有这些数据子句（present 除外）将在数据区域的开头分配设备内存，并在数据区域的结尾释放设备内存。除了创建之外，它们还会在主机和设备之间传输一定量的数据。

您还可以使用它们一次对多个数组进行操作，方法是将这些数组作为逗号分隔的列表包含在内。

```fortran
!$acc data copy( A(1:N), B(1:M), C(1:Q) )
```

您还可以一次使用多个数据子句。

```fortran
!$acc data create( A(1:N) ) copyin( B(1:M) ) copyout( C(1:Q) )
```

这些子句也可以直接与并行或内核指令一起使用，因为每个并行和内核指令都被**隐含数据区域**所包围。

```fortran
!$acc kernels create(A(1:N)) copyin(B(1:M)) present(C(1:Q))

    < Code that uses A, B, and C >

```

### 包含多个计算区域

单个数据区域可以包含任意数量的并行/内核区域。以下是示例：

```fortran
!$acc data copyin(A, B) create(C)

    !$acc parallel loop
    for i=1,N
        C(i) = A(i) + B(i)
    enddo
    
    !$acc parallel loop
    do i=1,N
        A(i) = C(i) + B(i)
    enddo

```

您还可以在数据区域内包含函数调用：

```fortran
subroutine copy(A, B, N)
integer :: A(:), B(:), N

!$acc parallel loop copyout(A) copyin(B)
do i=1,N
   A(i) = B(i)
enddo

end subroutine copy

...

!$acc data copyout(A,B) copyin(C)
call copy(A, C, N);
    
call copy(A, B, N);
!$acc end data
```

但是，这段代码现在不会导致数组被复制两次吗？一次由数据区域复制，另一次由函数调用中的并行循环复制？事实上，OpenACC 运行时足够智能，可以处理这种情况。数据只会在第一次遇到数据子句时被复制进来，而只有在最后一次遇到数据子句时才会被复制出去。这允许您在函数中创建完全有效的指令，然后在完全不更改代码的情况下将数据移动“提升”到更高的级别。这是逐步加速代码以避免错误结果的一部分。

### 数组整形

*数组形状*定义数组的一部分。以下是示例：

```fortran
integer, dimension(:), allocatable :: A
allocate( A(N) )

!$acc data create( A )
```

数组形状未定义，这意味着 GPU 副本将从索引 1 开始，大小为 N（即整个数组）。但是，您可以使用 A(2:N-1) 指定数组的子集。数组形状的格式为 **Array(starting_index:ending_index)** 。让我们看一个我们只想要数组的一部分的示例。

```fortran
integer, dimension(:), allocatable :: A
allocate( A(N) )

!$acc data create( A(1:N/2) )
```

在此示例中，GPU 副本将从索引 1 开始，但大小仅为 CPU 副本的一半。

多维数组的形状可以定义如下：

```fortran
!$acc data create( A(1:N,1:M) )
```

如果您未包含起始索引，则假定为 1。例如：

```fortran
!$acc data create( A(1:N) )
```

等同于

```fortran
!$acc data create( A(:N) )
```

在 Fortran 中，如果您未指定任何索引，**starting_index** 或 **ending_index** ，则编译器将使用 CPU 上数组的起始和结束索引假定您指的是整个数组。例如：

```fortran
!$acc data create( A )
```

### 主机或设备内存？

这里有两个循环：

```fortran
integer, dimension(:), allocatable :: A
allocate( A(N) )

do i=1,N
    A(i) = 0
enddo

!$acc parallel loop
do i=1,N
    A(i) = 1
enddo
```

第一个循环不包含在 OpenACC 计算区域内（计算区域由 parallel 或 kernels 指令标记）。因此， `A(i)` 将访问主机（CPU）内存。

第二个循环前面是 *parallel 指令*，这意味着它包含在 OpenACC 计算区域内。第二个循环中的 `A(i)` 将访问设备（GPU）内存。

### 将结构化数据指令添加到代码中

使用以下链接编辑我们的 laplace 代码。添加结构化数据指令以正确处理数组` A` 和` Anew` 。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

然后，运行以下脚本来检查您的解决方案。您的代码应该运行得与我们的托管内存代码一样好（或略好）。

In [ ]:
!nvfortran -fast -ta=tesla -Minfo=accel -o laplace_structured laplace2d.f90 jacobi.f90 && ./laplace_structured

如果您遇到困难，或者想检查答案，您可以通过以下链接查看正确答案。

[jacobi.f90](solutions/advanced_data/structured/jacobi.f90)

### 可选：分析代码

In [ ]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_structured ./laplace_structured

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并 <mark>右键单击</mark> [此处](laplace_structured.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看分析器报告，请参阅 [如何查看报告](../../../module2/assets/Fortran/README.ipynb) 部分。

花点时间探索分析器，当您准备好时，让我们放大分析器的开头。

![structured.PNG](../images/structured.png)

我们可以看到，我们有不间断的计算，并且我们所有的数据移动都发生在程序的开头。这是理想的，因为我们避免在计算过程中进行数据传输。

---

## OpenACC 非结构化数据指令

有两个非结构化数据指令：

**enter data**：处理设备内存分配，以及从主机到设备的复制。您可以与 `enter data` 一起使用的两个子句是 `create`（用于设备内存分配）和 `copyin`（用于分配和内存复制）。

**exit data**：处理设备内存释放，以及从设备到主机的复制。您可以与 `exit data` 一起使用的两个子句是 `delete`（用于设备内存释放），和 `copyout`（用于释放和内存复制）。

非结构化数据指令不标记“数据区域”，因为您可以在代码中拥有多个 `enter data` 和 `exit data` 指令。最好将它们纯粹视为内存分配和释放。

使用非结构化数据指令的最大优势是它们能够跨多个函数分支。您可以在一个函数中分配数据，并在另一个函数中释放数据。我们可以看一个简单的例子：

```fortran
subroutine intallocate(A,N)
    integer :: N
    integer, allocatable, dimension(:) :: A

    allocate( A(1:N) )
    !$acc enter data create (A(1:N))
end subroutine intallocate

subroutine intdeallocate(A,N)
    integer :: N
    integer, allocatable, dimension(:) :: A

    !$acc exit data delete(A(1:N))
    if (allocated(A)) deallocate(A)
end subroutine intdeallocate


program main
    integer, allocatable, dimension(:) :: A

    call intallocate(A,100)
    
    !$acc parallel loop
    do i=1,100
        A(i) = 0
    enddo
    
    call intdeallocate(A,100)
end program main
```

就像上面的代码示例一样，您必须先分配数组的 CPU 副本，然后才能分配 GPU 副本。此外，您必须先释放数组的 GPU，然后才能释放 CPU 副本。

### 向代码添加非结构化数据指令

我们将编辑代码以使用非结构化数据指令来处理内存管理。首先，运行以下脚本将代码重置为添加结构化数据指令之前的状态。

In [ ]:
!cp ./solutions/basic_data/jacobi.f90 ./jacobi.f90 && cp ./solutions/basic_data/laplace2d.f90 ./laplace2d.f90 && echo "Reset Finished"

现在编辑代码以使用非结构化数据指令。要充分利用非结构化数据指令，请尝试仅通过更改 **laplace2d.f90** 代码来使代码正常工作。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

运行以下脚本来检查您的解决方案。您的代码应该运行得与我们的结构化实现一样快。

In [ ]:
!nvfortran -fast -ta=tesla -Minfo=accel -o laplace_unstructured laplace2d.f90 jacobi.f90 && ./laplace_unstructured

如果您遇到困难，或者想检查答案，您可以通过以下链接查看正确答案。

[jacobi.f90](solutions/advanced_data/unstructured/jacobi.f90)

### 可选：分析代码

In [ ]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_unstructured ./laplace_unstructured

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并 <mark>右键单击</mark> [此处](laplace_unstructured.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看分析器报告，请参阅 [如何查看报告](../../../module2/assets/Fortran/README.ipynb) 部分。

花点时间探索分析器，当您准备好时，让我们放大分析器的开头。

![unstructured.PNG](../images/unstructured.png)

我们可以看到，我们有不间断的计算，并且我们所有的数据移动都发生在程序的开头。这是理想的，因为我们避免在计算过程中进行数据传输。如果您还对代码的结构化版本进行了性能分析，您会注意到性能分析几乎相同。这并不奇怪，因为结构化和非结构化方法在硬件层面的工作方式非常相似。但是，结构化数据区域在简单代码中可能更容易，而使用非结构化方法时，某些代码可能会运行得更好。程序员可以通过分析和性能分析来确定使用哪种方法。

---

## OpenACC 更新指令

当我们使用数据指令时，程序员可以在两个地方在主机和设备之间传输数据。对于结构化数据指令，我们有机会在区域的开始和结束处传输数据。对于非结构化数据指令，我们可以在使用输入数据和退出数据指令时传输数据。

但是，在您的程序中，有时您可能需要在数据区域的中间或在输入数据和退出数据之间传输数据。为了在这些时候传输数据，我们可以使用 `update` 指令。更新指令将明确在主机和设备之间传输数据。`update` 指令有两个子句：

**self**：self 子句将数据从设备传输到主机（GPU 到 CPU）
**device**：device 子句将数据从主机传输到设备（CPU 到 GPU）

语法如下：

`!$acc update self(A)`

`!$acc update device(A)`

所有数组整形规则均适用。

例如，让我们创建一个 laplace 代码版本，其中我们希望在循环每 100 次迭代后打印数组 **A**。代码如下所示：

```fortran
!$acc data copyin(A(n,m), Anew(n,m))
do while ( error .gt. tol .and. iter .lt. iter_max )
    error = calcNext(A, Anew, m, n)
    swap(A, Anew, m, n)
    
    if(mod(iter,100).eq.0 ) then
        write(*,'(i5,f10.6)'), iter, error
        do i=1,n
            do j=1,m
                write(*,'(f10.2)', advance="no"), A(i,j)
            enddo
            write(*,*) ' '
        enddo
    endif

    iter = iter + 1
enddo
```

让我们运行此代码（在一个非常小的数据集上，这样我们就不会通过打印数千个数字而使控制台过载）。

注意：您必须编辑 jacobi.f90 并将问题的维度从 4096 更改为 10。否则，代码将需要很长时间才能运行，并且会在屏幕上产生大量输出。

**再次**，请将问题的维度 `n` 和 `m` 更改为 10。jacobi.f90 中的原始行将如下所示。

```fortran
integer, parameter :: n=4096, m=4096, iter_max=1000
```

jacobi.f90 中的这一行现在需要看起来像下面这样：

```fortran
integer, parameter :: n=10, m=10, iter_max=1000
```

原因很简单，Fortran 代码没有命令行参数功能，而 C 版本有。单击下面的链接将允许您编辑代码。

[jacobi.f90](update/jacobi.f90)

[laplace2d.f90](update/laplace2d.f90)

In [ ]:
!nvfortran -fast -ta=tesla -Minfo=accel -o laplace_no_update ./update/laplace2d.f90 ./update/jacobi.f90 && ./laplace_no_update

我们可以看到数组没有变化。这是因为 `A` 的主机副本在循环迭代之间没有被**更新**。让我们添加更新指令，看看输出如何变化。

```fortran
!$acc data copyin( A(n,m), Anew(n,m))

do while ( error .gt. tol .and. iter .lt. iter_max )
    error = calcNext(A, Anew, m, n)
    swap(A, Anew, m, n)
    
    if(mod(iter,100).eq.0 ) then
        write(*,'(i5,f10.6)'), iter, error
        
        !$acc update self(A(n,m))
        do i=1,n
            do j=1,m
                write(*,'(f10.2)', advance="no"), A(i,j)
            enddo
        enddo
    endif
    iter = iter+1
    
enddo
```

注意：您必须编辑 jacobi.f90 并将问题的维度从 4096 更改为 10。否则，代码将需要很长时间才能运行，并且会在屏幕上产生大量输出。

**再次**，请将问题的维度 `n` 和 `m` 更改为 10。jacobi.f90 中的原始行将如下所示。

```fortran
integer, parameter :: n=4096, m=4096, iter_max=1000
```

jacobi.f90 中的这一行现在需要看起来像下面这样：

```fortran
integer, parameter :: n=10, m=10, iter_max=1000
```

原因很简单，Fortran 代码没有命令行参数功能，而 C 版本有。单击下面的链接将允许您编辑代码。

[jacobi.f90](update/solution/jacobi.f90)

[laplace2d.f90](update/solution/laplace2d.f90)

In [ ]:
!nvfortran -fast -ta=tesla -Minfo=accel -o laplace_update ./update/solution/laplace2d.f90 ./update/solution/jacobi.f90 && ./laplace_update

---

## 总结

依靠托管内存来处理数据管理可以减少程序员并行化代码所需的工作量，但是，并非所有 GPU 都使用托管内存，而且它的性能也低于使用显式数据管理。OpenACC 为程序员提供了两种处理数据管理的主要方法，即结构化和非结构化数据指令。通过使用这些，程序员能够最大限度地减少程序中所需的数据传输次数。

---

## 额外任务

如果您想要一些关于使用 OpenACC 的额外课程，OpenACC YouTube 页面上有一个 OpenACC 简介视频系列。该系列的第五个视频涵盖了本实验中涵盖的很多内容。

[OpenACC 并行编程简介 - 第 5 部分](https://youtu.be/0zTX7-CPvV8)

---

## 实验后总结

如果您想下载此实验以供以后查看，建议您转到浏览器的文件菜单（而不是 Jupyter 笔记本文件菜单）并保存完整的网页。这将确保图像也被复制下来。

您还可以执行以下单元块来创建您正在处理的文件的 zip 文件，并使用下面的链接下载它。

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**执行上述 zip 命令后**，您应该能够通过按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](openacc_files.zip) 并选择*将链接另存为*来下载并保存 zip 文件。

# 许可

本材料由 NVIDIA Corporation 根据知识共享署名 4.0 国际 (CC BY 4.0) 发布。